The overall idea is that we will implement linear and tree based models first and try to ansamble them. Further will implement recurrent NN and compare the two models.
Data normalisation required for linear or NN models will be done with the help of sklearn pipeline which will include both motel and data transofrmation steps. 

In [5]:
# TODO: try to train on log of target to smoothen extreme values

In [2]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, ElasticNet, Lasso, Ridge
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
import numpy as np

from src.ToyModel import ToyModel
from src.utilities import run_cv
from src.FeatureGenerator import FeatureGenerator

# CV data initiation

In [3]:
# initiating data generators
feat_generator = FeatureGenerator()

# initiating cv splitter
tscv = TimeSeriesSplit(test_size = 1, max_train_size=16)

# generating all of the data which we will iterate over during CV
features_df = feat_generator.generate_features()

# creating col lists for training
cols_di={
    'index': feat_generator.index_cols,
    'target': feat_generator.target_col,
    'feats': feat_generator.shifted_cols + feat_generator.roll_cols
}


# Baseline

In [3]:
# initiating toy model
toy_model = ToyModel()

# iterating over CV folds
toy_cv_res = run_cv(df=features_df, months_cv_split=tscv, model=toy_model, cols_di=cols_di, verbose=2)

Fold 0:
  Train months: [13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28], size: 120,494
  Test months: [29],   size: 7,039
  NRMSE:  1.0
  RMSE :  5.1

Fold 1:
  Train months: [14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29], size: 119,696
  Test months: [30],   size: 6,739
  NRMSE:  1.0
  RMSE :  4.4

Fold 2:
  Train months: [15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30], size: 118,557
  Test months: [31],   size: 5,669
  NRMSE:  1.0
  RMSE :  4.4

Fold 3:
  Train months: [16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31], size: 116,851
  Test months: [32],   size: 5,439
  NRMSE:  1.0
  RMSE :  4.9

Fold 4:
  Train months: [17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32], size: 114,617
  Test months: [33],   size: 5,668
  NRMSE:  1.0
  RMSE :  3.9


------------------------------
RMSE mean: 4.6
NRMSE mean: 1.0


# Linear model

In [5]:
# initiating lin model
lin_model =  Pipeline([
    ("scaler", StandardScaler()),
    ("lin_model", LinearRegression()) #ElasticNet() Lasso(alpha=.005) , 
])

# iterating over CV folds
lin_cv_res = run_cv(df=features_df, months_cv_split=tscv, model=lin_model, cols_di=cols_di, verbose=2)

Fold 0:
  Train months: [13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28], size: 120,494
  Test months: [29],   size: 7,039
  NRMSE:  0.5
  RMSE :  2.6

Fold 1:
  Train months: [14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29], size: 119,696
  Test months: [30],   size: 6,739
  NRMSE:  0.49
  RMSE :  2.2

Fold 2:
  Train months: [15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30], size: 118,557
  Test months: [31],   size: 5,669
  NRMSE:  0.4
  RMSE :  1.7

Fold 3:
  Train months: [16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31], size: 116,851
  Test months: [32],   size: 5,439
  NRMSE:  0.69
  RMSE :  3.4

Fold 4:
  Train months: [17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32], size: 114,617
  Test months: [33],   size: 5,668
  NRMSE:  0.71
  RMSE :  2.7


------------------------------
RMSE mean: 2.5
NRMSE mean: 0.56


# Tree based

In [4]:
# parameters found through 50 rounds of optuna optimisation
params = {'max_depth': 7, 'learning_rate': 0.005298559699829036, 'n_estimators': 500, 'gamma': 0.7744615794844047, 
          'reg_alpha': 4.7427786108127865, 'reg_lambda': 3.9464923699087713, 'colsample_bytree': 0.5556916732637939}

tree_model = XGBRegressor(**params)

tree_cv_res = run_cv(df=features_df, months_cv_split=tscv, model=tree_model, cols_di=cols_di, verbose=2)

Fold 0:
  Train months: [13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28], size: 120,319
  Test months: [29],   size: 6,857
  NRMSE:  0.41
  RMSE :  2.1

Fold 1:
  Train months: [14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29], size: 119,185
  Test months: [30],   size: 6,751
  NRMSE:  0.42
  RMSE :  1.9

Fold 2:
  Train months: [15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30], size: 118,117
  Test months: [31],   size: 5,763
  NRMSE:  0.39
  RMSE :  1.7

Fold 3:
  Train months: [16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31], size: 116,489
  Test months: [32],   size: 5,357
  NRMSE:  0.65
  RMSE :  3.2

Fold 4:
  Train months: [17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32], size: 114,199
  Test months: [33],   size: 5,598
  NRMSE:  0.55
  RMSE :  2.1


------------------------------
RMSE mean: 2.2
NRMSE mean: 0.49


# Recurrent NN